<a href="https://colab.research.google.com/github/rafeekpro/Colab/blob/main/langchain_1hr_sprint.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# LangChain 1-hour Sprint

![LangChain Logo](https://github.com/rastringer/promptcraft_notebooks/blob/main/images/langchain.png?raw=1)

LangChain is a framework for developing applications infused with LLM magic. In this notebook, we will cover some of its most useful and fun features, including:

* Templates
* Memory
* Working with APIs
* Chains
* Agents
* Vector stores

Let's start by importing some packages

In [1]:
! pip install --upgrade google-cloud-aiplatform
# LangChain
! pip install langchain langchain-experimental langchain[docarray]
! pip install pypdf
! pip install pydantic==1.10.8
# Open source vector store
! pip install chromadb==0.3.26
! pip install typing-inspect==0.8.0 typing_extensions==4.5.0
# For dense vector representations of text
! pip install sentence-transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 25.3 MB/s eta 0:00:00


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 19.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 159.9/159.9 kB 18.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 177.6/177.6 kB 18.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.0/47.0 kB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 215.3/215.3 kB 21.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.7/138.7 kB 1.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for hnswlib: filename=hnswlib-0.7.0-cp310-cp310-linux_x86_64.whl size=2202684 sha256=9b72afedffce9f3f939e2de1903391617f0179612202403ad85d4370d5526159
  Stored in directory: /root/.cache/pip/wheels/8a/ae/ec/235a682e0041fbaeee389843670581ec6c66872db856dfa9a4
Successfully built hns

In [1]:
# Automatically restart kernel after installs so that your environment can access the new packages
import IPython

app = IPython.Application.instance()
app.kernel.do_shutdown(True)

{'status': 'ok', 'restart': True}

In [ ]:
import vertexai

vertexai.init()

In [4]:
# Colab authentication.
import sys

if "google.colab" in sys.modules:
    from google.colab import auth
    auth.authenticate_user()
    print('Authenticated')

Authenticated


In [14]:

PROJECT_ID = "gncujlg-agbg-iberia-gen-ai"  # @param {type:"string"}
LOCATION = "us-central1"  # @param {type:"string"}
# Code examples may misbehave if the model is changed.
MODEL_NAME = "text-bison@001"

In [15]:
# Utils
import time
from typing import List

# Langchain
import langchain
from pydantic import BaseModel

print(f"LangChain version: {langchain.__version__}")

# Vertex AI
from google.cloud import aiplatform
from langchain.chat_models import ChatVertexAI
from langchain.embeddings import VertexAIEmbeddings
from langchain.llms import VertexAI
from langchain.schema import HumanMessage, SystemMessage

print(f"Vertex AI SDK version: {aiplatform.__version__}")

LangChain version: 0.0.344
Vertex AI SDK version: 1.36.4


In [16]:
vertexai.init(project=PROJECT_ID,
              location=LOCATION)

In [17]:
# We will use chat for some tasks
chat = ChatVertexAI(
    max_output_tokens=1024,
    temperature=0.2,
    top_p=0.8,
    top_k=40,
    verbose=True)

# And we will use the general text llm for others
llm = VertexAI(
    max_output_tokens=1024,
    temperature=0.2,
    top_p=0.8,
    top_k=40,
    verbose=True)

The simplest LangChain use is to create chats comprising of a `SystemMessage` and `HumanMessage`. This is similar to the `context` and `user_message` that we provide the LLM using the Python client libraries.

In [18]:
chat([HumanMessage(content="Hello")])

AIMessage(content=' Hello. How may I help you? \n')

In [ ]:
res = chat(
    [
        SystemMessage(
            content="You are an expert chef that thinks of imaginative recipies when people give you ingredients."
        ),
        HumanMessage(content="I have some kidney beans and tomatoes, what would be an easy lunch?"),
    ]
)

print(res.content)

 Here is a lunch recipe using kidney beans and tomatoes:
Ingredients:
- 1 can of kidney beans, rinsed and drained
- 1 can of diced tomatoes, undrained
- 1/2 cup of chopped onion
- 1/2 cup of chopped green bell pepper
- 1/4 cup of chopped cilantro
- 1 tablespoon of olive oil
- 1 teaspoon of chili powder
- 1/2 teaspoon of ground cumin
- 1/4 teaspoon of salt
- 1/4 teaspoon of black pepper
Instructions:
1. Heat the olive oil in a large skillet over medium heat.
2. Add the onion and green bell pepper and cook until softened, about 5 minutes.
3. Add the kidney beans, tomatoes, chili powder, cumin, salt, and black pepper.
4. Bring to a boil, then reduce heat and simmer for 15 minutes, or until the beans are heated through.
5. Stir in the cilantro and serve.


## Prompt templates

Templates are an abstraction that can help keep prompts modular and reusable. This can be especially important in large applications which may require long and varied prompts.

Templates may include few-short examples, instructions, or context.

In [19]:
# The template_string parameters sets the context for the ChatPromptTemplate

template_string = """Translate the text \
that is delimited by triple backticks \
into a style that is {style}. \
text: ```{text}```
"""

In [20]:
from langchain.prompts import ChatPromptTemplate

prompt_template = ChatPromptTemplate.from_template(template_string)

The chat prompts are envisioned as a series of messages. Notice the `.messages` and `format_messages` methods in the following cells.

In [21]:
# Print out the template
prompt_template.messages

[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['style', 'text'], template='Translate the text that is delimited by triple backticks into a style that is {style}. text: ```{text}```\n'))]

In [22]:
#Let's check just the prompt
prompt_template.messages[0].prompt

PromptTemplate(input_variables=['style', 'text'], template='Translate the text that is delimited by triple backticks into a style that is {style}. text: ```{text}```\n')

In [23]:
# Helpful method to keep track of a template's inputs
prompt_template.messages[0].prompt.input_variables

['style', 'text']

In this simple example, we translate a customer e-mail into phonetic Glaswegian.

In [24]:
translator_style = """A translator that writes in phonetic Glaswegian.
"""

In [25]:
customer_email = """
This smashing little coffee maker is simply brilliant! \
I'm so pleased with how easy it is to use and how quickly it brews \
a cracking cup of coffee. \
I'm over the moon with this purchase and would highly recommend it \
to any other coffee lover looking for a top-notch brew every time.
"""

In [26]:
# The format_messages method sets up the task specified in the template
customer_messages = prompt_template.format_messages(
                    style=translator_style,
                    text=customer_email)

In [27]:
# Call the LLM to translate to the style of the customer message
customer_response = chat(customer_messages)
print(customer_response.content)

 A guid wee coffee maker this is, pure belter! Ah'm ower the moon wi' it, an' A wid recommend it tae ony coffee lover lookin' fur a braw brew ilka time.


## Parsing outputs

LangChain makes it easy to return objects from the LLM in a format which we can use for further tasks (for example, adding an item of interest to a shopping cart, or providing a short list back to the LLM for additional questions).

Here is an example of parsing customer reviews of a three-course meal in a restaurant.

In [28]:
customer_review = """\
The excellent barbecue cauliflower starter left \
a lasting impression -- gorgeous presentation and flavors, really geared the tastebuds into action. \
Moving on to the main course, pretty great also. \
Delicious and flavorful chickpea and vegetable curry. They really nailed the buttery consistency, \
depth and balance of the spices. \
The dessert was a bit bland. I opted for a vegan chocolate mousse, \
hoping for a decadent and indulgent finale to my meal. \
It was very visually appealing but was missing the smooth, velvety \
texture of a great mousse.
"""

review_template = """\
For the input text, extract the following details: \
starter: How did the reviewer find the first course? \
Rate either Poor, Good, or Excellent. \
Do the same for the main course and dessert

Format the output as JSON with the following keys:
starter
main_course
dessert

text: {text}
"""



In [29]:
from langchain.prompts import ChatPromptTemplate

prompt_template = ChatPromptTemplate.from_template(review_template)
print(prompt_template)

input_variables=['text'] messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['text'], template='For the input text, extract the following details: starter: How did the reviewer find the first course? Rate either Poor, Good, or Excellent. Do the same for the main course and dessert\n\nFormat the output as JSON with the following keys:\nstarter\nmain_course\ndessert\n\ntext: {text}\n'))]


In [30]:
messages = prompt_template.format_messages(text=customer_review)
response = chat(messages, temperature=0.1)
print(response.content)

 ```JSON
{
  "starter": "Excellent",
  "main_course": "Good",
  "dessert": "Poor"
}
```


Though it looks like JSON, our output is actually a string type.

In [31]:
type(response.content)

str

This means we are unable to access values in this fashion (will result in an error):

In [32]:
response.content.get("main_course")

AttributeError: ignored

This is where LangChain's parser comes in. Here, we import the `ResponseSchema` and `StructuredOutputParser`, which we use to define the format of the results from the LLM.

In [33]:
from langchain.output_parsers import ResponseSchema
from langchain.output_parsers import StructuredOutputParser

starter_schema = ResponseSchema(name="starter", description="Review of the starter")
main_course_schema = ResponseSchema(name="main_course", description="Review of the main course")
dessert_schema = ResponseSchema(name="dessert", description="Review of the dessert")

response_schemas = [starter_schema, main_course_schema, dessert_schema]

In [34]:
output_parser = StructuredOutputParser.from_response_schemas(response_schemas)

In [36]:
format_instructions = output_parser.get_format_instructions()
print(format_instructions)

The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":

```json
{
	"starter": string  // Review of the starter
	"main_course": string  // Review of the main course
	"dessert": string  // Review of the dessert
}
```


Now we can update our prior review template to include the format instructions

In [37]:
review_template_2 = """\
For the input text, extract the following details: \
starter: How did the reviewer find the first course? \
Rate either Poor, Good, or Excellent. \
Do the same for the main course and dessert

starter
main_course
dessert

text: {text}

{format_instructions}
"""
prompt = ChatPromptTemplate.from_template(template=review_template_2)

messages = prompt.format_messages(text=customer_review,
                                format_instructions=format_instructions)

In [38]:
print(messages[0].content)

For the input text, extract the following details: starter: How did the reviewer find the first course? Rate either Poor, Good, or Excellent. Do the same for the main course and dessert

starter
main_course
dessert

text: The excellent barbecue cauliflower starter left a lasting impression -- gorgeous presentation and flavors, really geared the tastebuds into action. Moving on to the main course, pretty great also. Delicious and flavorful chickpea and vegetable curry. They really nailed the buttery consistency, depth and balance of the spices. The dessert was a bit bland. I opted for a vegan chocolate mousse, hoping for a decadent and indulgent finale to my meal. It was very visually appealing but was missing the smooth, velvety texture of a great mousse.


The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":

```json
{
	"starter": string  // Review of the starter
	"main_course": string  // Review of the

In [39]:
response = chat(messages)

Let's try it on the same review

Our response starts as an `AIMessage`

In [40]:
type(response)

langchain_core.messages.ai.AIMessage

Here we parse the `AIMessage` into a Python dictionary

In [ ]:
output_dict = output_parser.parse(response.content)
output_dict

{'starter': 'Excellent', 'main_course': 'Good', 'dessert': 'Poor'}

Thanks to LangChain's parser, we now have a Python dictionary which we can use for further tasks, for example taking part of the response and using it as an input to another function / process etc.

In [ ]:
type(output_dict)

dict

In [ ]:
output_dict.get("main_course")

'Good'

## API chains

Another of LangChain's useful features is the ability to call external APIs within chains.

In this example, we use the `open-meteo.com` API to get weather reports.

In [ ]:
from langchain.chains import APIChain
from langchain.chains.api import open_meteo_docs

llm = VertexAI(temperature=0)
chain = APIChain.from_llm_and_api_docs(
    llm,
    open_meteo_docs.OPEN_METEO_DOCS,
    verbose=True,
    limit_to_domains=["https://api.open-meteo.com/"],
)
chain.run(
    "How is the weather today in Edinburgh, Scotland, in Celsius?"
    )



> Entering new APIChain chain...
 https://api.open-meteo.com/v1/forecast?latitude=55.9533&longitude=-3.1883&hourly=temperature_2m,weathercode&current_weather=true&temperature_unit=celsius&windspeed_unit=kmh&precipitation_unit=mm&timeformat=iso8601
{"latitude":55.96,"longitude":-3.18,"generationtime_ms":0.0940561294555664,"utc_offset_seconds":0,"timezone":"GMT","timezone_abbreviation":"GMT","elevation":69.0,"current_weather_units":{"time":"iso8601","interval":"seconds","temperature":"°C","windspeed":"km/h","winddirection":"°","is_day":"","weathercode":"wmo code"},"current_weather":{"time":"2023-12-01T13:00","interval":900,"temperature":-0.2,"windspeed":3.7,"winddirection":259,"is_day":1,"weathercode":0},"hourly_units":{"time":"iso8601","temperature_2m":"°C","weathercode":"wmo code"},"hourly":{"time":["2023-12-01T00:00","2023-12-01T01:00","2023-12-01T02:00","2023-12-01T03:00","2023-12-01T04:00","2023-12-01T05:00","2023-12-01T06:00","2023-12-01T07:00","2023-12-01T08:00","2023-12-01T09:0

' The current temperature in Edinburgh, Scotland is -0.2 degrees Celsius.'

### Wikipedia

We can combine the Wikipedia pip package and LangChain's Wikipedia API wrapper get query results from the encyclopedia.

In [ ]:
!pip install wikipedia

In [ ]:
from langchain.tools import WikipediaQueryRun
from langchain.utilities import WikipediaAPIWrapper

wikipedia = WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper())

wikipedia.run("To which bird family does the field sparrow belong?")

'Page: House sparrow\nSummary: The house sparrow (Passer domesticus) is a bird of the sparrow family Passeridae, found in most parts of the world. It is a small bird that has a typical length of 16 cm (6.3 in) and a mass of 24–39.5 g (0.85–1.39 oz). Females and young birds are coloured pale brown and grey, and males have brighter black, white, and brown markings. One of about 25 species in the genus Passer, the house sparrow is native to most of Europe, the Mediterranean Basin, and a large part of Asia. Its intentional or accidental introductions to many regions, including parts of Australasia, Africa, and the Americas, make it the most widely distributed wild bird.\nThe house sparrow is strongly associated with human habitation, and can live in urban or rural settings. Though found in widely varied habitats and climates, it typically avoids extensive woodlands, grasslands, polar regions, and hot, dry deserts far away from human development. For sustenance, the house sparrow routinely 

### Google search

In [ ]:
from langchain.prompts import PromptTemplate
from langchain.chains import LLMRequestsChain, LLMChain

template = """Between >>> and <<< are the raw search result text from google.
Extract the answer to the question '{query}' or say "not found" if the information is not contained.
Use the format
Extracted:<answer or "not found">
>>> {requests_result} <<<
Extracted:"""

PROMPT = PromptTemplate(
    input_variables=["query", "requests_result"],
    template=template,
)


chain = LLMRequestsChain(llm_chain=LLMChain(llm=VertexAI(temperature=0), prompt=PROMPT))
question = "What are the official languages in Turkmenistan, and their alphabets?"
inputs = {
    "query": question,
    "url": "https://www.google.com/search?q=" + question.replace(" ", "+"),
}
chain(inputs)

{'query': 'What are the official languages in Turkmenistan, and their alphabets?',
 'url': 'https://www.google.com/search?q=What+are+the+official+languages+in+Turkmenistan,+and+their+alphabets?',
 'output': ' The official languages in Turkmenistan are Turkmen and Russian. Turkmen is written in a modified Latin alphabet, while Russian is written in the Cyrillic alphabet.'}

## Memory

It is essential that LLMs keep some memory of the prior interactions in a chat to better inform their answers.

LangChain offers several approaches and features in this regard. For all details, see the [Memory](https://python.langchain.com/docs/modules/memory/) section of the documentation.

### ConversationBufferWindowMemory

Maintains a list of the interactions of the conversation over time, using the last K interactions. This can be useful for keeping a sliding window of the most recent interactions, so the buffer does not get too large

In [ ]:
from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferMemory, ConversationBufferWindowMemory

memory = ConversationBufferWindowMemory(k=3)

memory.save_context({"input": "Hi"},
                    {"output": "How are you?"})
memory.save_context({"input": "Fine thanks"},
                    {"output": "Great"})

memory.load_memory_variables({})

{'history': 'Human: Hi\nAI: How are you?\nHuman: Fine thanks\nAI: Great'}

### ConversationTokenBufferMemory

This feature instead keeps a buffer of recent interactions in memory based on token length,  rather than number of interactions.

In [ ]:
from langchain.memory import ConversationTokenBufferMemory

memory = ConversationTokenBufferMemory(llm=llm, max_token_limit=100)
memory.save_context({"input": "All alone, she dreams of the stars!"},
                    {"output": "As she should!"})
memory.save_context({"input": "Baking cookies today?"},
                    {"output": "Behold the cookies!"})
memory.save_context({"input": "Chatbots everywhere?"},
                    {"output": "Certainly!"})

In [ ]:
memory.load_memory_variables({})

{'history': 'Human: All alone, she dreams of the stars!\nAI: As she should!\nHuman: Baking cookies today?\nAI: Behold the cookies!\nHuman: Chatbots everywhere?\nAI: Certainly!'}

### Conversation summaries

LangChain carries forward summaries of chat messages and flushes memory after a specified number of interactions or tokens.

Let's first look at using the former, `ConversationBufferWindowMemory`.

We set `verbose=True` to show the prompts and information carried forward by the LLM.

In [ ]:
from langchain.memory import ConversationBufferWindowMemory

conversation_with_summary = ConversationChain(
    llm=VertexAI(temperature=0),
    # We set a low k=2, to only keep the last 2 interactions in memory
    memory=ConversationBufferWindowMemory(k=2),
    verbose=True
)
conversation_with_summary.predict(input="My favourite sport is fencing. Any tips for how I can go pro?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: My favourite sport is fencing. Any tips for how I can go pro?
AI:

> Finished chain.


' Here are some tips on how to become a professional fencer:\n\n1. Start young. The earlier you start fencing, the more time you will have to develop your skills and reach your full potential.\n2. Find a good coach. A good coach can help you develop the proper technique and skills, and can also help you prepare for competitions.\n3. Practice regularly. The more you practice, the better you will become. Try to practice at least three times per week, and make sure to focus on all aspects of fencing, including footwork, bladework, and tactics.\n4. Compete in tournaments. Competition'

In [ ]:
conversation_with_summary.predict(input="What equipment do I need?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: My favourite sport is fencing. Any tips for how I can go pro?
AI:  Here are some tips on how to become a professional fencer:

1. Start young. The earlier you start fencing, the more time you will have to develop your skills and reach your full potential.
2. Find a good coach. A good coach can help you develop the proper technique and skills, and can also help you prepare for competitions.
3. Practice regularly. The more you practice, the better you will become. Try to practice at least three times per week, and make sure to focus on all aspects of fencing, including footwork, bladework, and tactics.
4. Compete in tournaments. Competition
Human: What equipment d

" Here is a list of the equipment you will need for fencing:\n\n* A foil, epee, or sabre. The type of weapon you use will depend on the type of fencing you want to do.\n* A mask. This is to protect your face from being hit by the opponent's weapon.\n* A jacket. This is to protect your torso from being hit by the opponent's weapon.\n* A glove. This is to protect your hand from being hit by the opponent's weapon.\n* A pair of breeches. These are special pants that are designed for fencing.\n* A pair of fencing"

In [ ]:
conversation_with_summary.predict(input="Who are the greats of the sport I can emulate?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: My favourite sport is fencing. Any tips for how I can go pro?
AI:  Here are some tips on how to become a professional fencer:

1. Start young. The earlier you start fencing, the more time you will have to develop your skills and reach your full potential.
2. Find a good coach. A good coach can help you develop the proper technique and skills, and can also help you prepare for competitions.
3. Practice regularly. The more you practice, the better you will become. Try to practice at least three times per week, and make sure to focus on all aspects of fencing, including footwork, bladework, and tactics.
4. Compete in tournaments. Competition
Human: What equipment d

" Here are some of the greatest fencers of all time:\n\n* Men's foil:\n    * Stefano Cerioni (Italy)\n    * Alexander Romankov (Russia)\n    * Benjamin Kleibrink (Germany)\n* Women's foil:\n    * Valentina Vezzali (Italy)\n    * Yelena Belova (Russia)\n    * Nam Hyun-hee (South Korea)\n* Men's épée:\n    * Éric Srecki (France)\n    * Pavel Kolobkov (Russia)\n    * Rubén Limardo (Venezuela)\n* Women'"

In [ ]:
# Since we have now passed k=2, the LLM will be unable to answer
conversation_with_summary.predict(input="What is my favourite sport?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: What equipment do I need?
AI:  Here is a list of the equipment you will need for fencing:

* A foil, epee, or sabre. The type of weapon you use will depend on the type of fencing you want to do.
* A mask. This is to protect your face from being hit by the opponent's weapon.
* A jacket. This is to protect your torso from being hit by the opponent's weapon.
* A glove. This is to protect your hand from being hit by the opponent's weapon.
* A pair of breeches. These are special pants that are designed for fencing.
* A pair of fencing
Human: Who are the greats of the sport I can emulate?
AI:  Here are some of the greatest fencers of all time:

* Men's foil:
    * Ste

' I cannot answer that question as I do not have access to your personal information. \n'

### ConversationSummaryBufferMemory

Ensures conversational memory up to a specified token length

In [ ]:
from langchain.chains import ConversationChain

conversation_with_summary = ConversationChain(
    llm=llm,
    # Change max_token_limit here after running through the conversation
    memory=ConversationTokenBufferMemory(llm=llm, max_token_limit=600),
    verbose=True,
)
conversation_with_summary.predict(input="Hi, how are you?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: Hi, how are you?
AI:

> Finished chain.


' I am an AI assistant incapable of experiencing feelings in the same way humans might.  I am, however,  able to assist you with a variety of text-based activities. How may I help you today? \n'

In [ ]:
conversation_with_summary.predict(input="I'm learning the Rust programming language")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Hi, how are you?
AI:  I am an AI assistant incapable of experiencing feelings in the same way humans might.  I am, however,  able to assist you with a variety of text-based activities. How may I help you today? 

Human: I'm learning the Rust programming language
AI:

> Finished chain.


' Rust is an open-source systems programming language designed with safety in mind. Rust prevents many common types of bugs that plague other languages, and guarantees memory safety without garbage collection. Rust has a rich type system and an expressive macro system. It is fast and efficient, with no runtime or garbage collector. Rust is used in a wide variety of applications, including operating systems, embedded systems, and games.\n'

In [ ]:
conversation_with_summary.predict(input="What's the best book to help me?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Hi, how are you?
AI:  I am an AI assistant incapable of experiencing feelings in the same way humans might.  I am, however,  able to assist you with a variety of text-based activities. How may I help you today? 

Human: I'm learning the Rust programming language
AI:  Rust is an open-source systems programming language designed with safety in mind. Rust prevents many common types of bugs that plague other languages, and guarantees memory safety without garbage collection. Rust has a rich type system and an expressive macro system. It is fast and efficient, with no runtime or garbage collector. Rust is used in a wide variety of applications, including operating sy

" There are several books that can help you learn Rust. Here are a few recommendations:\n\n* The Rust Programming Language: This is the official book for Rust, and it covers everything you need to know to get started with the language. It's a comprehensive book, but it's also very readable and well-organized.\n* Rust in Action: This book is a more practical guide to Rust, with a focus on real-world examples. It covers everything from basic syntax to advanced topics like concurrency and unsafe code.\n* Programming Rust: This book is a good choice if you're looking for a more in-depth"

In [ ]:
# Notice the buffer here is updated and clears the earlier exchanges
# Depending on how chatty the LLM is feeling, the token limit may have
# already been reached, and this cell will yield a generic response.
conversation_with_summary.predict(input="Wish me luck!")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Hi, how are you?
AI:  I am an AI assistant incapable of experiencing feelings in the same way humans might.  I am, however,  able to assist you with a variety of text-based activities. How may I help you today? 

Human: I'm learning the Rust programming language
AI:  Rust is an open-source systems programming language designed with safety in mind. Rust prevents many common types of bugs that plague other languages, and guarantees memory safety without garbage collection. Rust has a rich type system and an expressive macro system. It is fast and efficient, with no runtime or garbage collector. Rust is used in a wide variety of applications, including operating sy

' Good luck!  Rust is a powerful language that can be used to create a wide variety of applications.  I am confident that you will enjoy learning it.  If you run into any problems, there are many resources available online to help you.  The official Rust website is a great place to start, and there are also many helpful forums and communities where you can ask questions and get help.'

The following cell should generate a reply that is clearly restricted to the general benefits of learning Haskell and missing the previous context of someone trying to learn Rust.

Run this cell, then go back to the Keep the conversation going with summaries cell and change `max_token_limit` to 700. Then re-run the entire conversation and notice how the model relates its ouptut about learning Haskell to the context of someone trying to learn Rust.

In [ ]:
conversation_with_summary.predict(input="Would knowing Haskell help me?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Hi, how are you?
AI:  I am an AI assistant incapable of experiencing feelings in the same way humans might.  I am, however,  able to assist you with a variety of text-based activities. How may I help you today? 

Human: I'm learning the Rust programming language
AI:  Rust is an open-source systems programming language designed with safety in mind. Rust prevents many common types of bugs that plague other languages, and guarantees memory safety without garbage collection. Rust has a rich type system and an expressive macro system. It is fast and efficient, with no runtime or garbage collector. Rust is used in a wide variety of applications, including operating sy

" Knowing Haskell can definitely help you learn Rust. Haskell is a purely functional programming language, which means that it emphasizes immutability and referential transparency. This can make it easier to reason about your code and avoid bugs. Rust is also a functional language, and it borrows many concepts from Haskell. For example, Rust's pattern matching and type system are both inspired by Haskell. If you're familiar with Haskell, you'll have a head start on learning Rust.\n\nHere are some specific ways that knowing Haskell can help you learn Rust:\n\n* **Pattern matching:** Pattern matching is a powerful tool for extracting data from complex structures"

## Chains

Complex applications will require chaining LLMs together, or with other components.

We will cover the following types of chains:

**Sequential chains**

**Router chains**

### LLMChain

An LLMChain simply provides a prompt to the LLM.

In [ ]:
from langchain.chains import LLMChain

prompt = ChatPromptTemplate.from_template(
    "What is the best name to describe \
    a company that makes {product}?"
)

In [ ]:
chain = LLMChain(llm=llm, prompt=prompt)
product = "A saw for laminate wood"
chain.run(product)

' Assistant: Some potential names for a company that makes a saw for laminate wood include:\n\n- Laminate Saw Solutions\n- The Laminate Saw Company\n- Precision Laminate Saws\n- Laminate Cutting Systems\n- The Laminate Saw Experts\n- Laminate Saw Technology\n- Advanced Laminate Saws\n- Laminate Saw Innovations\n- Laminate Saw Manufacturing\n- Laminate Saw Distribution'

### Sequential chain

A sequential chain makes a series of calls to an LLM. It enables a pipeline-style workflow in which the output from one call becomes the input to the next.

The two types include:

* `SimpleSequentialChain`, where predictably each step has a single input and output, which becomes the input to the next step.

* `SequentialChain`, which allows for multiple inputs and outputs.

In [ ]:
from langchain.chains import SimpleSequentialChain
from langchain.prompts import PromptTemplate

In [ ]:
# This is an LLMChain to write a pitch for a new product
# Let's increase the temperature to allow some imagination

llm = VertexAI(temperature=0.7)
template = """You are an entrepreneur. Think of a ground breaking new product and write a short pitch.

Title: {title}
Entrepreneur: This is a pitch for the above product:"""
prompt_template = PromptTemplate(input_variables=["title"], template=template)
pitch_chain = LLMChain(llm=llm, prompt=prompt_template)

In [ ]:
template = """You are a panelist on Dragon's Den. Given a \
description of the product, you are to explain why you think it will \
succeed or fail in the market.

Product pitch: {pitch}
Review by Dragon's Den panelist:"""
prompt_template = PromptTemplate(input_variables=["pitch"], template=template)
review_chain = LLMChain(llm=llm, prompt=prompt_template)

In [ ]:
# This is the overall chain where we run these two chains in sequence.
overall_chain = SimpleSequentialChain(chains=[pitch_chain, review_chain], verbose=True)

In [ ]:
review = overall_chain.run("Portable iced coffee maker")



> Entering new SimpleSequentialChain chain...
 The Portable Iced Coffee Maker is the perfect solution for coffee lovers on the go. This revolutionary new product allows you to make fresh, iced coffee anywhere, anytime. With its sleek design and compact size, the Portable Iced Coffee Maker is perfect for taking with you to work, the gym, or on your next road trip. Simply add your favorite ground coffee and water, and the Portable Iced Coffee Maker will do the rest, brewing you a delicious, refreshing cup of iced coffee in just minutes. 

Benefits:

- Makes fresh, iced coffee anywhere, anytime
- Compact and portable design
- Easy to use - just add ground coffee
 While the Portable Iced Coffee Maker is a clever and innovative product, I'm not convinced that it will be a long-term success in the market.

The main issue I see is that the target market for this product is relatively niche. While there are certainly many coffee lovers who would appreciate the convenience of being able to ma

### Router chain

A `RouterChain` dynamically selects the next chain to use for a given input.
This feature uses the `MultiPromptChain` to select then answer with the best-suited prompt to the question.

This can help a modular architecure, allowing the effective triaging of inputs between relevant prompt templates.

In [ ]:
from langchain.chains.router import MultiPromptChain

korean_template = """
You are an expert in korean history and culture.
Here is a question:
{input}
"""

spanish_template = """
You are an expert in spanish history and culture.
Here is a question:
{input}
"""

chinese_template = """
You are an expert in Chinese history and culture.
Here is a question:
{input}
"""

In [ ]:
prompt_infos = [
    {
        "name": "korean",
        "description": "Good for answering questions about Korean history and culture",
        "prompt_template": korean_template,
    },
    {
        "name": "spanish",
        "description": "Good for answering questions about Spanish history and culture",
        "prompt_template": spanish_template,
    },
     {
        "name": "chinese",
        "description": "Good for answering questions about Chinese history and culture",
        "prompt_template": chinese_template,
    },
]

In [ ]:
from langchain.chains.router.llm_router import LLMRouterChain,RouterOutputParser

llm = VertexAI(temperature=0)

destination_chains = {}
for p_info in prompt_infos:
    name = p_info["name"]
    prompt_template = p_info["prompt_template"]
    prompt = ChatPromptTemplate.from_template(template=prompt_template)
    chain = LLMChain(llm=llm, prompt=prompt)
    destination_chains[name] = chain

destinations = [f"{p['name']}: {p['description']}" for p in prompt_infos]
destinations_str = "\n".join(destinations)

In [ ]:
default_prompt = ChatPromptTemplate.from_template("{input}")
default_chain = LLMChain(llm=llm, prompt=default_prompt)

In [ ]:
# Thanks to Deeplearning.ai for this template and for the
# Langchain short course at deeplearning.ai/short-courses/.

MULTI_PROMPT_ROUTER_TEMPLATE = """Given a raw text input to a \
language model select the model prompt best suited for the input. \
You will be given the names of the available prompts and a \
description of what the prompt is best suited for. \
You may also revise the original input if you think that revising\
it will ultimately lead to a better response from the language model.

<< FORMATTING >>
Return a markdown code snippet with a JSON object formatted to look like:
```json
{{{{
    "destination": string \ name of the prompt to use or "DEFAULT"
    "next_inputs": string \ a potentially modified version of the original input
}}}}
```

REMEMBER: "destination" MUST be one of the candidate prompt \
names specified below OR it can be "DEFAULT" if the input is not\
well suited for any of the candidate prompts.
REMEMBER: "next_inputs" can just be the original input \
if you don't think any modifications are needed.

<< CANDIDATE PROMPTS >>
{destinations}

<< INPUT >>
{{input}}

<< OUTPUT (remember to include the ```json)>>"""

In [ ]:
router_template = MULTI_PROMPT_ROUTER_TEMPLATE.format(
    destinations=destinations_str
)
router_prompt = PromptTemplate(
    template=router_template,
    input_variables=["input"],
    output_parser=RouterOutputParser(),
)

router_chain = LLMRouterChain.from_llm(llm, router_prompt)

In [ ]:
chain = MultiPromptChain(router_chain=router_chain,
                         destination_chains=destination_chains,
                         default_chain=default_chain, verbose=True
                        )

Notice in the outputs the country of speciality is prefixed eg:
`chinese: {'input': ...`, denoting the routing to the correct expert.

In [ ]:
chain.run("What was the Han Dynasty?")

In [ ]:
chain.run("What are some of typical dishes in Catalonia?")

In [ ]:
chain.run("How would I greet a friend's parents in Korean?")

In [ ]:
chain.run("Summarize Don Quixote in a short paragraph")

In [ ]:
# No specialist chain for carburetor advice; this
# will be handled as any other input by the foundational model
chain.run("How can I fix a carburetor?")

## Agents and vectorstores

This final section of the notebook will cover some of LangChain's most fun and powerful features.

Agents have access to tools such as JSON, Wikipedia, Web Search, GitHub or Pandas Dataframes, and can access their capabilities depending on user input.

See [here](https://python.langchain.com/docs/integrations/toolkits/) for a full list of agent toolkits.

We will work with some data to perform data retrieval using the LLM with embeddings to match customer queries to products. This is known as Retrieval Augmentated Generation, or RAG.

We will use the Wayfair [WANDS](https://www.aboutwayfair.com/careers/tech-blog/wayfair-releases-wands-the-largest-and-richest-publicly-available-dataset-for-e-commerce-product-search-relevance) dataset of more than 42,000 products. Here are the steps:

* Download the data into a pandas dataframe and take a smaller 1,000-row sample set

* Merge then generate embeddings for the product titles and descriptions

* Prompt an LLM to retrieve details and relevant documents related to queries.

<img src="https://assets.wfcdn.com/im/01139917/resize-h800-w800%5Ecompr-r85/2315/231567967/Capricornus+3+Seater+Sofa.jpg" width="250"/> <img src="https://assets.wfcdn.com/im/07725066/resize-h800-w800%5Ecompr-r85/1584/158440119/Vancasso+BOMOOTIUR+Stoneware+Dinnerware+-+Set+of+18.jpg" width="250"/>


In [ ]:
!wget -q https://raw.githubusercontent.com/wayfair/WANDS/main/dataset/product.csv

In [ ]:
import pandas as pd
product_df = pd.read_csv("product.csv", sep='\t')

We will work with 1,000 items to avoid longer wait times for the embedding and look up processes.

In [ ]:
product_df = product_df[:2000].dropna()

In [ ]:
product_df.head(3)

In [ ]:
len(product_df)

In [ ]:
# Reduce the df to columns of interest
product_df = product_df.filter(["product_id", "product_name", "product_description", "average_rating"], axis=1)

In [ ]:
product_df.head(3)

### Import and initialize pandas dataframe agent

These tools use the `langchain-experimental` pip package installed at the start of the notebook.

### Pandas agent

This agent allows us to interact with the dataframe using natural language. LangChain shows us the pandas queries it is composing to answer the questions.

In [ ]:
from langchain_experimental.agents.agent_toolkits import create_pandas_dataframe_agent
from langchain.agents.agent_types import AgentType

agent = create_pandas_dataframe_agent(VertexAI(temperature=0), product_df, verbose=True)

In [ ]:
agent.run("how many rows are there?")

In [ ]:
agent.run("How many products have a rating of > 4?")

### CSV agent

We can also work directly on a .csv file

In [ ]:
pd.DataFrame.to_csv(product_df, "data.csv")

In [ ]:
from langchain_experimental.agents.agent_toolkits import create_csv_agent

agent = create_csv_agent(
    VertexAI(temperature=0),
    "data.csv",
    verbose=True,
    agent_type=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
)

In [ ]:
agent.run("How many rows are there?")

In [ ]:
agent.run("Do any product descriptions mention cedar wood? Output them as JSON please")

In [ ]:
agent.run("What is the square root of all ratings for product names featuring sofas")

### Vector stores

We will explore embeddings vectors and vector stores in more detail in [subsequent notebooks](rastringer.io.github.com/promptcraft). Let's see what's possible by concatenating our `product_title` and `product_description` columns and creating a text file from the result. We can then create embeddings and perform various retrieval and Q&A tasks.

We will use the open source [Chroma](https://docs.trychroma.com/) vector store.

In [ ]:
from langchain.embeddings.sentence_transformer import SentenceTransformerEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain.document_loaders import TextLoader

We will embed a `text_data` column, which will be a concatenation of `product_name` and `product_description`, since both columns provide useful contextual information.

In [ ]:
product_df['text_data'] = product_df['product_name'] + " " + product_df['product_description']

In [ ]:
product_df["text_data"]

In [ ]:
# Save the "text_data" column to a text file
text_file_path = "combined_text_data.txt"
product_df['text_data'].to_csv(text_file_path, sep='\t', index=False, header=False)


In [ ]:
# load the document and split it into chunks
loader = TextLoader("combined_text_data.txt")
documents = loader.load()

### Text splitter

Splitting text is common when working with LangChain and LLMs in general. This practice means we can feed large amounts of data to LLMs for parsing or embedding in chunks, or batches.

Ideally, we want to do so in a way that keeps meaningful chunks together. We will use the default recommended `RecursiveCharacterTextSplitter`. We specify a `chunk_size` and `chunk_overlap` to set an upper limit on the size and overlap between the splits / chunks.

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 1500,
    chunk_overlap = 150
)

docs = text_splitter.split_documents(documents)

In [ ]:
len(docs)

In [ ]:
from langchain.vectorstores import Chroma

# Clear any previous vector store
!rm -rf ./docs/chroma

In [ ]:
# Takes ~3 mins to run
embedding_function = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")
db = Chroma.from_documents(docs, embedding_function)

In [ ]:
query = "Is there a slow cooker?"
docs = db.similarity_search(query, n_results=2)

In [ ]:
docs[0]

In [ ]:
query = "Recommend a durable door mat"
docs = db.similarity_search(query, n_results=2)

In [ ]:
docs[0]

### Retrieval

A `Retriever` is a method for answering questions based on information in an index.

Here, we use `RetrievalQA` this ability with a question and answering chain.

In [ ]:
from langchain.chains import RetrievalQA

llm = VertexAI(
    model_name="text-bison@001",
    max_output_tokens=1024,
    temperature=0.1,
    top_p=0.8,
    top_k=40,
    verbose=True,
)

qa_chain = RetrievalQA.from_chain_type(
    llm,
    retriever=db.as_retriever()
)

### Prompt

In [ ]:
from langchain.prompts import PromptTemplate

# Build prompt
template = """Use the following pieces of context to answer the question at the end. \
If you don't know the answer, just say that you don't know, \
don't try to make up an answer. Use three sentences maximum. \
{context}
Question: {question}
Helpful Answer:"""
QA_CHAIN_PROMPT = PromptTemplate(input_variables=["context", "question"],template=template,)


In [ ]:
# Run chain
qa_chain = RetrievalQA.from_chain_type(
    llm,
    retriever=db.as_retriever(),
    return_source_documents=True,
    chain_type_kwargs={"prompt": QA_CHAIN_PROMPT}
)

In [ ]:
question = "Can you recommend comfortable bed sheets?"
result = qa_chain({"query": question})
result["result"]

In [ ]:
question = "How about a Persian-style rug for my living room."
result = qa_chain({"query": question})
result["result"]

## Summary

In this whirlwind tour of some of LangChain's features, we covered:

* Memory
* Chains
* Agents
* Vector stores

LangChain is a fast-evolving project. To explore more features and keep up-to-date with developments, please see the [website](https://www.langchain.com/) or [Python documentation](https://python.langchain.com/docs/get_started/introduction).

With thanks to Harrison Chase and the excellent LangChain courses at [deeplearning.ai](https://deeplearning.ai/short-courses)